- 각 store에서 units이 모두 0인 item_nbr은 날린다..
    - 계산량을 줄이기 위해서
    


In [ ]:
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression

In [1]:
key = pd.read_csv('../data/key.csv')
train = pd.read_csv('../data/train.csv')
weather = pd.read_csv('../data/weather.csv')
test = pd.read_csv('../data/test.csv')

In [ ]:
weather_ = weather.drop(['date', 'codesum'], axis = 1)

In [ ]:
weather_.at[0, 'sunrise']

In [ ]:
col_list = list(weather_.columns)
col_list = col_list[1:]

In [ ]:
for num in range(len(weather_)) :
    for col in col_list :
        if 'M' in weather_.at[num, col] or 'T' in weather_.at[num, col] or '-' in weather_.at[num, col] :
            weather_.set_value(num, col, np.nan)

In [3]:
def TM_transform(series, T_replace, M_replace): 
    """
    데이터내의 T, M을 원하는 값으로 바꿔주는 함수
    TM_transform(series, T_replace, M_replace)
    """
    series = series.astype(str).map(lambda s: s.strip())
    series[series == 'T'] = T_replace
    series[series =='M'] = M_replace
    return series.astype('float')

In [4]:
def match_dateformat(df, year):
    """
    영문 월을 숫자 월로 바꾸어주고 나중에 사용하기 쉽도록 datetime.date 형태로 바꾸어주는 함수
    """
    months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
    for i in range(len(df)):
        dates = df.loc[i][0]
        dates = dates.split(" ")
        for j in range(len(months)):
            if dates[0] == months[j]:
                dates[0] = str(j + 1)
                dates_df = ["{} {} {}".format(year, dates[0], dates[1])]
                dates_df = pd.to_datetime(dates_df)
                df.loc[i][0] = dates_df.date[0]
    return df

In [5]:
def merge_holiday(holiday_df1, holiday_df2, holiday_df3):
    """
    각 연도별 공휴일 리스트 합치기
    """
    frame = [holiday_df1, holiday_df2, holiday_df3]
    holiday = pd.concat(frame).reset_index(drop=True)
    return holiday

In [6]:
def find_holiday(file, year):
    """
    수요에 영향을 미치는 주요 공휴일을 찾아내는 함수
    """
    holidays = ["New Year's Day", "Martin Luther King Jr. Day", "Valentine's Day",  "President's Day", "Easter Sunday", 
                      "Mother's Day", "Memorial Day", "Father's Day", "Independence Day", "Labor Day", "Columbus Day",
                      "Halloween", "Veterans Day", "Thanksgiving Day", "Black Friday", "Christmas Eve", "Christmas Day", "New Year's Eve"]
    
    holi = pd.read_excel(file, year, header=None)
    holi = match_dateformat(holi, year)
    holiday = pd.DataFrame(columns=[0,1,2,3,4])
    for _ in holidays:
        for i in range(len(holi[2])):
            if _ == holi[2][i]:
                holiday = holiday.append(holi.loc[i])
    return holiday

In [7]:
def preprocessing(df, holiday):
    """
    train데이터를 가공하는 함수
    """
#     df['units'] = np.log(df['units'] + 1)
    df['date'] = pd.to_datetime(df['date'])
    df['weekday'] = df.date.dt.weekday  # 월요일이 0 일요일이 6
    df['weekend'] = df.date.dt.weekday.isin([5, 6])  # 5: 토요일, 6: 일요일

    df['holiday'] = df.date.isin(holiday[0])
    df['weekday_holiday'] = df.holiday & (df.weekend == False)
    df['weekend_holiday'] = df.holiday & df.weekend

    # There are values that are computed as being "normal" for a given location. Normal are computed based on 30 years worth of data every ten years. (today temp - normal) = depart
    weather['date'] = pd.to_datetime(weather['date']) #weather는 글로벌변수
    for i in range(len(weather['codesum'])):
        codesum = weather['codesum'][i].split(" ")
        for _ in codesum:
            if _ == 'RA':
                weather.set_value(i, 'rain_flag', 1)
            elif _ == 'SN':
                weather.set_value(i, 'snow_flag', 1)
            elif _ == "":
                weather.set_value(i, 'normal_flag', 1)
            else:
                weather.set_value(i, 'abnormal_flag', 1)

    # return x or y depending on the condition
    # "For the purposes of this competition, we have defined a weather event as any day in which more than an inch of rain or two inches of snow was observed."
    # weather_event =  (((codesum contains SN) and (snowfall > 2)) or ((codesum contains RA) and (preciptotal > 1)))
    weather['preciptotal'] = TM_transform(weather['preciptotal'], 0.005, 0.00)
    weather['snowfall'] = TM_transform(weather['snowfall'], 0.005, 0.00)
    weather['snow_event'] = np.where(np.where(weather['snow_flag'] == 1, 1, 0) + np.where(weather['snowfall'] > 2, 1, 0) == 2, 1, 0)
    weather['rain_event'] = np.where(np.where(weather['rain_flag'] == 1, 1, 0) + np.where(weather['preciptotal'] > 1, 1, 0) == 2, 1, 0)
    weather['event'] = weather['snow_event'] + weather['rain_event']
    

#    weather['preciptotal_flag'] = np.where(weather['preciptotal'] > 0.2, 1, 0)
#     weather['depart'] = TM_transform(weather['depart'], np.nan, 0.00)
#     weather['depart_flag'] = np.where(weather['depart'] > 8.0, 1, 0)
#     weather['depart_flag'] = np.where(weather['depart'] < 8.0, -1, 0)
    
    df = pd.merge(df, key, on='store_nbr') #key는 글로벌 변수
    
    df = pd.merge(df, weather[['date', 'station_nbr', 'rain_flag', 'snow_flag', 'normal_flag', 'event']], on=['date', 'station_nbr'])
    return df

In [9]:
def preprocessing2(df, holiday):
    """
    train데이터를 가공하는 함수
    """
#     df['units'] = np.log(df['units'] + 1)
    df['date'] = pd.to_datetime(df['date'])
    df['weekday'] = df.date.dt.weekday  # 월요일이 0 일요일이 6
    df['weekend'] = df.date.dt.weekday.isin([5, 6])  # 5: 토요일, 6: 일요일

    df['holiday'] = df.date.isin(holiday[0])

    # return x or y depending on the condition
    # "For the purposes of this competition, we have defined a weather event as any day in which more than an inch of rain or two inches of snow was observed."
    # weather_event =  (((codesum contains SN) and (snowfall > 2)) or ((codesum contains RA) and (preciptotal > 1)))
    weather['preciptotal'] = TM_transform(weather['preciptotal'], 0.005, np.NaN)
    weather['snowfall'] = TM_transform(weather['snowfall'], 0.005, np.NaN)
#     weather['snow_event'] = np.where(np.where(weather['snow_flag'] == 1, 1, 0) + np.where(weather['snowfall'] > 2, 1, 0) == 2, 1, 0)
#     weather['rain_event'] = np.where(np.where(weather['rain_flag'] == 1, 1, 0) + np.where(weather['preciptotal'] > 1, 1, 0) == 2, 1, 0)
    weather['event'] = ""
    
    for num in range(len(df)) :
        if weather.at[num, 'snowfall'] >= 2 or weather.at[num, 'preciptotal'] >= 1 :
            weather.set_value(num, 'event', 1)
        else :
            weather.set_value(num, 'event', 0)
    

#    weather['preciptotal_flag'] = np.where(weather['preciptotal'] > 0.2, 1, 0)
#     weather['depart'] = TM_transform(weather['depart'], np.nan, 0.00)
#     weather['depart_flag'] = np.where(weather['depart'] > 8.0, 1, 0)
#     weather['depart_flag'] = np.where(weather['depart'] < 8.0, -1, 0)
    
    df = pd.merge(df, key, on='store_nbr') #key는 글로벌 변수
    
    df = pd.merge(df, weather[['date', 'station_nbr', 'event']], on=['date', 'station_nbr'])
    return df

In [28]:
def preprocessing3(df, holiday):
    """
    train데이터를 가공하는 함수
    """
#     df['units'] = np.log(df['units'] + 1)
    df['date'] = pd.to_datetime(df['date'])
    df['weekday'] = df.date.dt.weekday  # 월요일이 0 일요일이 6
    df['weekend'] = df.date.dt.weekday.isin([5, 6])  # 5: 토요일, 6: 일요일

    df['holiday'] = df.date.isin(holiday[0])
    df['weekday_holiday'] = df.holiday & (df.weekend == False)
    df['weekend_holiday'] = df.holiday & df.weekend

    # There are values that are computed as being "normal" for a given location. Normal are computed based on 30 years worth of data every ten years. (today temp - normal) = depart
    weather['date'] = pd.to_datetime(weather['date']) #weather는 글로벌변수
    for i in range(len(weather['codesum'])):
        codesum = weather['codesum'][i].split(" ")
        for _ in codesum:
            if _ == 'RA':
                weather.set_value(i, 'rain_flag', 1)
            elif _ == 'SN':
                weather.set_value(i, 'snow_flag', 1)
            elif _ == "":
                weather.set_value(i, 'normal_flag', 1)
            else:
                weather.set_value(i, 'abnormal_flag', 1)

    # return x or y depending on the condition
    # "For the purposes of this competition, we have defined a weather event as any day in which more than an inch of rain or two inches of snow was observed."
    # weather_event =  (((codesum contains SN) and (snowfall > 2)) or ((codesum contains RA) and (preciptotal > 1)))
    weather['preciptotal'] = TM_transform(weather['preciptotal'], 0.005, 0.00)
    weather['snowfall'] = TM_transform(weather['snowfall'], 0.005, 0.00)
    weather['snow_event'] = np.where(np.where(weather['snow_flag'] == 1, 1, 0) + np.where(weather['snowfall'] > 2, 1, 0) == 2, 1, 0)
    weather['rain_event'] = np.where(np.where(weather['rain_flag'] == 1, 1, 0) + np.where(weather['preciptotal'] > 1, 1, 0) == 2, 1, 0)
    weather['event'] = weather['snow_event'] + weather['rain_event']
    
    for num in range(len(weather)) :
        if weather.at[num, 'event'] == 2 :
            weather.set_value(num, 'event', 1)
    

#    weather['preciptotal_flag'] = np.where(weather['preciptotal'] > 0.2, 1, 0)
#     weather['depart'] = TM_transform(weather['depart'], np.nan, 0.00)
#     weather['depart_flag'] = np.where(weather['depart'] > 8.0, 1, 0)
#     weather['depart_flag'] = np.where(weather['depart'] < 8.0, -1, 0)
    
    df = pd.merge(df, key, on='store_nbr') #key는 글로벌 변수
    
    df = pd.merge(df, weather[['date', 'station_nbr', 'rain_flag', 'snow_flag', 'normal_flag', 'event']], on=['date', 'station_nbr'])
    return df

In [ ]:
def test_preprocessing(df, holiday, weather):
    df['date'] = pd.to_datetime(df['date'])
    df['weekday'] = df.date.dt.weekday  # 월요일이 0 일요일이 6
    df['weekend'] = df.date.dt.weekday.isin([5, 6])  # 5: 토요일, 6: 일요일

    df['holiday'] = df.date.isin(holiday[0])
    df['weekday_holiday'] = df.holiday & (df.weekend == False)
    df['weekend_holiday'] = df.holiday & df.weekend
    
    df = pd.merge(df, key, on='store_nbr') #key는 글로벌 변수
    df = pd.merge(df, weather[['date', 'station_nbr', 'event']], on=['date', 'station_nbr'])
    return df

In [10]:
holiday12 = find_holiday('../data/holiday.xlsx', '2012')
holiday13 = find_holiday('../data/holiday.xlsx', '2013')
holiday14 = find_holiday('../data/holiday.xlsx', '2014')
holiday = merge_holiday(holiday12, holiday13, holiday14)
processed_train = preprocessing(train, holiday)

In [ ]:
processed_train.drop(['weekend', 'weekday_holiday', 'weekend_holiday', 'station_nbr',
                      'rain_flag', 'snow_flag', 'normal_flag'], axis = 1, inplace = True)

In [ ]:
processed_train.sort_values(by = ['date', 'store_nbr', 'item_nbr'], inplace = True)
processed_train.reset_index(drop = True, inplace = True)

In [ ]:
processed_train = processed_train[:2255853]

In [ ]:
processed_train.loc[0]

In [12]:
train['log_units'] = np.log(train['units'] + 1)
train.tail()

,date,store_nbr,item_nbr,units,weekday,weekend,holiday,weekday_holiday,weekend_holiday,log_units
4617595,2014-10-31,45,107,0,4,False,True,True,False,0.0
4617596,2014-10-31,45,108,0,4,False,True,True,False,0.0
4617597,2014-10-31,45,109,0,4,False,True,True,False,0.0
4617598,2014-10-31,45,110,0,4,False,True,True,False,0.0
4617599,2014-10-31,45,111,0,4,False,True,True,False,0.0


In [13]:
train_new = train[:2255853]
train_new.head()

,date,store_nbr,item_nbr,units,weekday,weekend,holiday,weekday_holiday,weekend_holiday,log_units
0,2012-01-01,1,1,0,6,True,True,False,True,0.0
1,2012-01-01,1,2,0,6,True,True,False,True,0.0
2,2012-01-01,1,3,0,6,True,True,False,True,0.0
3,2012-01-01,1,4,0,6,True,True,False,True,0.0
4,2012-01-01,1,5,0,6,True,True,False,True,0.0


In [ ]:
tmp = train_new.pivot_table(values = 'log_units', index = 'date', columns = ['item_nbr'])
item_nbr_total = list(tmp.columns)

In [ ]:
tmp = tmp.loc[:, (tmp != 0).any(axis = 0)]

In [ ]:
tmp_ = tmp.reset_index()

In [ ]:
tmp_.head()

In [ ]:
item_nbr_nonzero = list(tmp.columns)

In [ ]:
item_nbr_zero = []

for num in item_nbr_total :
    if num not in item_nbr_nonzero :
        item_nbr_zero.append(num)

In [ ]:
item_nbr_zero

In [ ]:
for num in item_nbr_zero :
    train_new_ = train_new[train_new.item_nbr != num[1]]

In [ ]:
train_new_.reset_index(drop = True, inplace = True)

In [ ]:
train_test = preprocessing(train_new_, holiday)

In [ ]:
train_test
train_test_ = train_test.drop(['weekend', 'weekday_holiday', 'weekend_holiday', 'station_nbr',
                               'rain_flag', 'snow_flag', 'normal_flag', 'units', 'date'], axis = 1)

In [ ]:
train_test_.head()

In [ ]:
train_test_['store_nbr'] = train_test_['store_nbr'].astype('category')
train_test_['item_nbr'] = train_test_['item_nbr'].astype('category')
train_test_['weekday'] = train_test_['weekday'].astype('category')
train_test_['holiday'] = train_test_['holiday'].astype('category')
train_test_['event'] = train_test_['event'].astype('category')

In [ ]:
model1 = sm.OLS.from_formula("log_units ~ store_nbr + item_nbr + weekday + holiday + event", data = train_test_)

In [ ]:
result1 = model1.fit()
print(result1.summary())

In [ ]:
model2 = sm.OLS.from_formula("log_units ~ store_nbr + item_nbr + weekday + holiday + event + 0", data = train_test_)

In [ ]:
result2 = model2.fit()
print(result2.summary())

In [ ]:
model3 = sm.OLS.from_formula("log_units ~ C(store_nbr) + C(item_nbr) + C(weekday) + C(holiday) + C(event)", data = train_test_)

In [ ]:
result3 = model3.fit()
print(result3.summary())

In [ ]:
model4 = sm.OLS.from_formula("log_units ~ C(store_nbr) + C(item_nbr) + C(weekday) + C(holiday) + C(event) + 0 + 0 + 0 + 0 + 0", data = train_test_)

In [ ]:
result4 = model4.fit()
print(result4.summary())

In [ ]:
model5 = sm.OLS.from_formula("log_units ~ C(item_nbr) + 0", data = train_test_)
result5 = model5.fit()
print(result5.summary())

In [22]:
tmp2 = train_new.pivot_table(values = 'log_units', index = ['date', 'store_nbr', 'item_nbr'])
tmp2_ = tmp2[tmp2['log_units'] != 0]

In [23]:
tmp2_ = tmp2_.reset_index()

In [19]:
tmp2_ = preprocessing(tmp2_, holiday)

In [29]:
tmp2_ = preprocessing3(tmp2_, holiday)

In [30]:
tmp2_.tail()

,date,store_nbr,item_nbr,log_units,weekday,weekend,holiday,weekday_holiday,weekend_holiday,station_nbr,rain_flag,snow_flag,normal_flag,event
61313,2013-03-30,35,24,1.945910,5,True,False,False,False,5,NaN,NaN,1.0,0
61314,2013-03-30,35,63,2.302585,5,True,False,False,False,5,NaN,NaN,1.0,0
61315,2013-03-30,35,66,3.637586,5,True,False,False,False,5,NaN,NaN,1.0,0
61316,2013-03-31,35,16,3.218876,6,True,True,False,True,5,NaN,NaN,1.0,0
61317,2013-03-31,35,66,2.890372,6,True,True,False,True,5,NaN,NaN,1.0,0


In [32]:
tmp2_test = tmp2_.drop(['weekend', 'weekday_holiday', 'weekend_holiday', 'station_nbr',
                               'rain_flag', 'snow_flag', 'normal_flag', 'date'], axis = 1)

In [ ]:
tmp2_test['store_nbr'] = tmp2_test['store_nbr'].astype('category')
tmp2_test['item_nbr'] = tmp2_test['item_nbr'].astype('category')
tmp2_test['weekday'] = tmp2_test['weekday'].astype('category')
tmp2_test['holiday'] = tmp2_test['holiday'].astype('category')
tmp2_test['event'] = tmp2_test['event'].astype('category')

In [ ]:
tmp2_test.head()

In [ ]:
tmp2_test_add = sm.add_constant(tmp2_test)

In [ ]:
tmp2_test_add.head()

In [ ]:
tmp2_test.head()

In [33]:
model = sm.OLS.from_formula("log_units ~ 0 + C(store_nbr) + C(item_nbr) + C(weekday) + C(holiday) + C(event)", data = tmp2_test)

In [34]:
result = model.fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:              log_units   R-squared:                       0.873
Model:                            OLS   Adj. R-squared:                  0.873
Method:                 Least Squares   F-statistic:                     2829.
Date:                Mon, 05 Mar 2018   Prob (F-statistic):               0.00
Time:                        10:54:37   Log-Likelihood:                -45406.
No. Observations:               61318   AIC:                         9.111e+04
Df Residuals:                   61168   BIC:                         9.247e+04
Df Model:                         149                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
C(store_nbr)[1]        1.2953      0

- p>t : 이 column의 계수가 0이다 : 귀무사걸 
- 기각하면 0이 아니다..constant가 0이 아니다..
- skew,kurtosis : 잔차의 skew, kurtosis
- prob(omnibus) 귀무가설의 잔차가 normal이다..
- jarque-bera : normality test.. 분포가 normal이 아니다...
- condition no : 다중공성성...크면 안좋다... 기준이 약 10000

In [ ]:
result.params[40:]

[22, 38, 47, 64, 95, 104, 105, 106, 107, 108, 109, 110, 111]

In [ ]:
test_sub = preprocessing(test, holiday)

In [ ]:
test_sub = test_sub.drop(['weekend', 'weekday_holiday', 'weekend_holiday', 'station_nbr',
                               'rain_flag', 'snow_flag', 'normal_flag', 'date'], axis = 1)

In [ ]:
test_sub.head()

In [ ]:
result.predict(test_sub)

In [ ]:
from patsy import dmatrix, demo_data, ContrastMatrix, Poly
data = demo_data("a", nlevels=3)

In [ ]:
data

In [ ]:
dmatrix("C(a) + 0", data)

In [ ]:
dmatrix("C(a)", data)

- test set을 만들어서 kaggle에 제출하여 결과 매니져님께..
- 1번카테고리가 날라가면 2번으로 대체하는 방식..? 비슷한걸로 방식..
- 평균 그래프에서 결론을 내는건 data loss가 있을 수 있어서 위험하다